In [1]:
!pip install transformers PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.5 MB/s eta 0:00:00


In [2]:
import os
import csv
import PyPDF2
from transformers import pipeline


In [3]:
from huggingface_hub import login

login()

In [4]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 3.3 MB/s eta 0:00:00


In [5]:
# Extract text from PDF
def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file."""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = "\n".join(page.extract_text() for page in reader.pages if page.extract_text())
        text = text.replace("\n", " ").strip()  # Remove all line breaks

    return text

In [6]:
# Chunking function
def chunk_text(text, max_chars=1000):
    """Split text into chunks of a specified maximum character length, removing line breaks."""
    chunks = []
    while len(text) > max_chars:
        split_point = text[:max_chars].rfind('.')  # Find the last period within the limit
        if split_point == -1:
            split_point = max_chars
        chunks.append(text[:split_point + 1].strip())
        text = text[split_point + 1:].strip()
    if text:
        chunks.append(text)
    return chunks

In [7]:
# Summarize content
def summarize_text(chunks, max_chars=1000):
    """Summarize text into a list of key sentences."""
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    summaries = []
    for chunk in chunks:
        summary = summarizer(chunk, max_length=50, min_length=20, do_sample=False)
        summaries.extend(s['summary_text'] for s in summary)
    return " ".join(summaries)

In [8]:
from groq import Groq

In [9]:
client = Groq(

    # This is the default and can be omitted

    api_key="Your API key here",

)

In [10]:
def generate_flashcards_from_summary(summary, client, model="llama3-8b-8192"):
    """Generate flashcard questions and answers from a summarized text using the specified API."""
    prompt = (
        """Generate 20 flashcards based on the key information in the following text.
        Ensure the flashcards include clear questions with concise and precise answers.
        Each flashcard should help the user quickly understand and remember the subject matter. Use the format below:
        Q: [Write a focused question]
        A: [Provide a clear, concise answer]
        The questions should cover all essential points and concepts from the text, including definitions, examples.
        Text: {summary}""".format(summary=summary)
    )

    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "you are a helpful assistant."
                },
                {
                    "role": "user",
                    "content": prompt
                },
            ],
            model=model,
        )
        return chat_completion.choices[0].message.content
    except Exception as e:
        print(f"Error generating flashcards: {e}")
        return None

In [11]:
def parse_flashcards(raw_flashcards):
    """Parse raw flashcard text into a list of question-answer pairs."""
    flashcards = []
    lines = raw_flashcards.splitlines()
    question, answer = None, None
    for line in lines:
        if line.startswith("Q:"):
            question = line[3:].strip()
        elif line.startswith("A:"):
            answer = line[3:].strip()
            if question and answer:
                flashcards.append((question, answer))
                question, answer = None, None
    return flashcards

def save_flashcards_to_csv(flashcards, output_file="flashcards.csv"):
    """Save flashcard questions and answers to a CSV file."""
    try:
        with open(output_file, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(["Question", "Answer"])
            for question, answer in flashcards:
                writer.writerow([question, answer])
    except Exception as e:
        print(f"Error saving flashcards to CSV: {e}")

In [12]:
def process_pdf_to_csv(client, output_csv_path="flashcards.csv"):
    """Extract text, process it into flashcards, and save to CSV."""
    pdf_path = input("Please enter the name of your PDF file (eg. example.pdf): ")
    print("Extracting text from PDF...")
    text = extract_text_from_pdf(pdf_path)
    print("Extraction done !")

    print("Chunking text...")
    chunks = chunk_text(text)
    print("Text is chunked into smaller piece !\n")


    print("Summarizing chunks...")
    summary = summarize_text(chunks)
    print("Your text has been summarized !\n")


    print("Generating flashcards from summary...")
    raw_flashcards = generate_flashcards_from_summary(summary, client)
    print("You have successfully generated your flashcards !\n")

    print("Parsing flashcards...")
    flashcards = parse_flashcards(raw_flashcards)
    print("Saving flashcards to CSV...")
    save_flashcards_to_csv(flashcards, output_csv_path)
    print("Flashcards saved to", output_csv_path)

In [13]:
process_pdf_to_csv(client)


Please enter the name of your PDF file (eg. example.pdf): A_Brief_Introduction_To_AI.pdf
Extracting text from PDF...
Extraction done !
Chunking text...
Text is chunked into smaller piece !

Summarizing chunks...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


Your text has been summarized !

Generating flashcards from summary...
You have successfully generated your flashcards !

Parsing flashcards...
Saving flashcards to CSV...
Flashcards saved to flashcards.csv
